In [ ]:
import cv2
import os
import numpy as np
from natsort import natsorted
import matplotlib.pyplot as plt
from PIL import Image

#import carla

In [ ]:
#basedir = '/home/tristram/data/carla/25_yes_bkgd/'
#outdir = '/home/tristram/data/carla/15_yes_bkgd/'

inp = np.load(basedir+'3D_boxes.npy', allow_pickle=True).item()
rep = {}
for keys in inp:
    ts, car, _ = keys.split('_')
    if int(ts) <= 15:
        rep[keys] = inp[keys]

np.save(outdir+'3D_boxes.npy', rep)

sky = np.load(basedir+'sky_masks.npz', allow_pickle=True)['arr_0']
sky_out = sky[:15]
np.savez(outdir+'sky_masks.npz', sky_out)

d = np.load(basedir+'depth_images.npz', allow_pickle=True)['arr_0']
d_out = d[:15]
np.savez(outdir+'depth_images.npz', d_out)

m = np.load(basedir+'2D_boxes.npz', allow_pickle=True)['arr_0']
m_out = m[:15]
np.savez(outdir+'2D_boxes.npz', m_out)

poses = np.load(basedir+'poses_bounds.npy')
p_out = poses[:15]
np.save(outdir+'poses_bounds.npy', p_out)

In [ ]:
basedir = '/home/tristram/data/carla/testbox/'
outdir = '/home/tristram/data/carla/100_sphere_mvmt_nobkgd/'

img_list = natsorted(os.listdir(basedir))

all_depth = []
for name in img_list: 
    if 'DEPTH' in name:
        d = cv2.imread(basedir+name)
        d = np.array(d[...,::-1])  # RGB Ordering from BGR

        normalized_depth = np.dot(d[:, :, :3], [1.0, 256.0, 65536.0])
        normalized_depth /= 16777215.0  # (256.0 * 256.0 * 256.0 - 1.0)
        in_meters = 1000 * normalized_depth
        all_depth.append(in_meters)
        
all_depth = np.array(all_depth)
print(len(all_depth))
#np.savez(outdir + 'depth_images.npz', all_depth)        

In [ ]:
import numpy as np
import cv2
from PIL import Image

def visualize_depth(depth, cmap=cv2.COLORMAP_TWILIGHT_SHIFTED):
    """
    depth: (H, W)
    """
    x = depth
    
    x = np.nan_to_num(x) # change nan to 0
    mi = np.min(x) # get minimum depth
    ma = np.max(x)
    x = (x-mi)/(ma-mi+1e-8) # normalize to 0~1
    x = (255*x).astype(np.uint8)
    x_ = Image.fromarray(cv2.applyColorMap(x, cmap))
    return x_

In [ ]:
basedir = '/home/tristram/data/carla/testbox/'

img_list = natsorted(os.listdir(basedir))
all_masks = []
for name in img_list:
    if 'SEM' in name:
        sem = cv2.imread(basedir+name)
        sky = 13
        mask = (sem[:,:,-1] == sky).astype(np.float32)
        all_masks.append(mask)
        
all_masks = np.array(all_masks)
print(len(all_masks))
np.savez(outdir + 'sky_masks.npz', all_masks)

In [ ]:
img = cv2.imread(basedir + '00196615_1_INS.png')
cv2.imshow('test', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
"""
Remove Background from images to learn bbox movement
"""
for name in img_list:
    if 'INS' in name:
        ins = cv2.imread(basedir+name)
        #ins = np.array(ins[..., ::-1])
        m1 = (ins[:, :, 0] == 204) & (ins[:, :, 1] == 72) & (ins[:, :, 2] == 10)
        inv = (m1 == False)
        ts, cam, _ = name.split('_')
        rgb = ts+'_'+cam+'_RGB.png'
        img = cv2.imread(basedir+rgb)
        #img = np.array(img[...,::-1])
        img[inv] = 0
        cv2.imwrite(outdir+'images_4/'+rgb, img)

## REMEMBER TO CHANGE RESOLUTION

In [ ]:
near = 3.0
far = 200.0
focal = 4.0 * 514.6816609

resolution = np.array([[1920, 1280], [1920, 1280]])

poses = np.load(outdir + 'poses.npy')

j = 0
poses_3x5 = []
for i in range(poses.shape[0]):
    if i > 0 and i % (2 + j * 5) == 0:
        hwf = np.array([resolution[1, 1], resolution[1,0], focal]).reshape(3,1)
        pose = poses[i, :3, :4]
        poses_3x5.append(np.hstack((pose, hwf)))
    elif i > 0 and i % (4 + j * 5) == 0:
        hwf = np.array([resolution[1, 1], resolution[1,0], focal]).reshape(3,1)
        pose = poses[i, :3, :4]
        poses_3x5.append(np.hstack((pose, hwf)))
    else:
        hwf = np.array([resolution[0, 1], resolution[0,0], focal]).reshape(3,1)
        pose = poses[i, :3, :4]
        poses_3x5.append(np.hstack((pose, hwf)))
        
    if i > 0 and i % 5 == 0:
        j += 1
        
poses_flat = np.array(poses_3x5).reshape(-1, 15)
poses_final = []
for elem in poses_flat:
    poses_final.append(np.concatenate((elem, [near, far])))
poses_final = np.array(poses_final)
print(poses_final.shape)
np.save(outdir+'poses_bounds.npy', poses_final)        

In [ ]:
boxes = np.load(basedir+'bboxes.npy', allow_pickle=True).item()
imgs = natsorted(os.listdir(outdir+'images_4/'))
raw_imgs = natsorted(os.listdir(basedir))

ins_list = []
for name in raw_imgs:
    if 'INS' in name:
        ins_list.append(name)
clean = {}
for key in boxes:
    xy = {}
    for key2 in boxes[key]:
        for key3 in boxes[key][key2]:
            if key3 not in 'extent':
                if key3 not in 'center':
                    xy[key+'_'+key3+'-'+key2] = boxes[key][key2][key3]
    for k in xy:
        ts, car = k.split('-')
        for ins in ins_list:
            if ts in ins:
                im = cv2.imread(basedir+ins)
                x_min = xy[k]['x_min']
                x_max = xy[k]['x_max']
                y_min = xy[k]['y_min']
                y_max = xy[k]['y_max']

                # check that box is containing a car and not background!
                if 10 in im[y_min:y_max, x_min:x_max, 2]:
                    
                    clean[k] = xy[k]
                    
                    cv2.line(im, (int(x_min), int(y_min)), (int(x_max), int(y_min)), (0, 0, 255, 255), 1)
                    cv2.line(im, (int(x_min), int(y_max)), (int(x_max), int(y_max)), (0, 0, 255, 255), 1)
                    cv2.line(im, (int(x_min), int(y_min)), (int(x_min), int(y_max)), (0, 0, 255, 255), 1)
                    cv2.line(im, (int(x_max), int(y_min)), (int(x_max), int(y_max)), (0, 0, 255, 255), 1)
                
                    #cv2.imshow("test", im)
                    #cv2.waitKey(800)
                    #cv2.destroyAllWindows()
                    
sorted_keys = natsorted(clean.keys())
masks_2d = []
for i in range(len(imgs)):
    img = cv2.imread(outdir+'images_4/'+imgs[i])[:,:,1]
    mask = np.zeros_like(img)
    for ks in sorted_keys:
        cam, car = ks.split('-')
        ts, cam = cam.split('_')
        tsi, ci, _ = imgs[i].split('_')
        if ts in tsi:
            if cam in ci:
                x_min = clean[ks]['x_min']
                x_max = clean[ks]['x_max']
                y_min = clean[ks]['y_min']
                y_max = clean[ks]['y_max']
                mask[y_min:y_max, x_min:x_max] = int(car)
    masks_2d.append(mask)

print(len(masks_2d))    
masks_2d = np.array(masks_2d)
np.savez(outdir + '2D_boxes.npz', masks_2d)

In [ ]:
clean = {}
cars = []
i = 1

for keys in boxes:
    for s in sorted_keys:
        ts, car = s.split('-')
        ts, c = ts.split('_')
        if car not in cars:
            cars.append(car)
        if ts in keys:
            clean[str(i)+'_'+car+'_center'] = boxes[ts][car]['center']
            clean[str(i)+'_'+car+'_extent'] = boxes[ts][car]['extent']
    i+=1


dict_3d = {}
for c in cars:
    for keys in clean:
        ts, car, cat = keys.split('_')
        if ts == '1' and car == c and cat == 'center':
            can_car = c
            can_pose = clean[ts+'_'+c+'_center']
            can_ext = clean[ts+'_'+c+'_extent']
            dict_3d[ts+'_'+c+'_center'] = can_pose
            #dict_3d[ts+'_'+c+'_rel'] = np.eye(4)
            dict_3d[ts+'_'+c+'_ext'] = can_ext
        elif car == c and cat == 'center':
            # we go from pose at ts x to can_pose and not the other way around!!
            rel_pose = np.matmul(can_pose, np.linalg.inv(clean[ts+'_'+c+'_center']))
            abs_pose = np.matmul(rel_pose, clean[ts+'_'+c+'_center'])
            #print(rel_pose, c)
            #print(clean[ts+'_'+c+'_center'])
            #print(c)
            #print(ts)
            dict_3d[ts+'_'+c+'_center'] = clean[ts+'_'+c+'_center']
            #dict_3d[ts+'_'+c+'_rel'] = rel_pose
            dict_3d[ts+'_'+c+'_ext'] = clean[ts+'_'+c+'_extent']

np.save(outdir+'3D_boxes.npy', dict_3d)

In [ ]:
basedir = '/home/tristram/Project/carla_0.9.13/CARLA_0.9.13/PythonAPI/examples/_out/'

img_list = natsorted(os.listdir(basedir))

ins_list = []
for name in img_list:
    if 'INS' in name:
        ins_list.append(name)

poses = np.load(outdir + 'poses.npy')

dyns = []
for j in range(5):
    for i in range(j, len(ins_list)-5, 5):
        if j == 0:
            distp = np.sqrt((poses[i, 2, 3] - poses[i+5, 2, 3])**2)
        else:
            distp = np.sqrt((poses[i, 2, 3] - poses[i+5, 2, 3])**2 +
                            (poses[i, 0, 3] - poses[i+5, 0, 3])**2)
            
        inst1 = np.array(Image.open(basedir+ins_list[i]))[:,:,:3]
        inst2 = np.array(Image.open(basedir+ins_list[i+5]))[:,:,:3]
        mask1 = (inst1[:,:,0] == 10)
        mask2 = (inst2[:,:,0] == 10)
        vehicle1 = inst1[mask1].copy()
        vehicle2 = inst2[mask2].copy()
        
        if vehicle1.sum() != 0 and vehicle2.sum() != 0:
            vehicle1 = np.unique(vehicle1[:,1:], axis=0)
            vehicle2 = np.unique(vehicle2[:,1:], axis=0)
            if vehicle1.shape == vehicle2.shape:
                for k, v in enumerate(vehicle1):
                    mask1 = ((inst1[:,:,1] == v[0]) & (inst1[:,:,2] == v[1]))
                    mask2 = ((inst2[:,:,1] == vehicle2[k, 0]) & (inst2[:,:,2] == vehicle2[k, 1]))
                    inv1 = (mask1 == False)
                    inv2 = (mask2 == False)
                    depth1 = all_depth[i].copy()
                    depth2 = all_depth[i+5].copy()
                    depth1[inv1] = 0.0
                    depth2[inv2] = 0.0
                    avg1 = np.mean(depth1[depth1>0.0])
                    avg2 = np.mean(depth2[depth2>0.0])
                    avgd = np.abs(avg1 - avg2)
                    diff = distp - avgd
                    if diff < 0.0 and (mask1.astype(np.float32).sum() > 100 or mask2.astype(np.float32).sum() > 100):
                         dyns.append(v)
dyns = np.array(dyns)
print(dyns)

In [ ]:
dyns = np.unique(dyns, axis=0)
dyn_masks = []
for ins in ins_list:
    instance = np.array(Image.open(basedir+ins))[:,:,:3]
    dynmask = np.zeros_like(instance[:,:,0])
    i = 1
    for d in dyns:
        mask = ((instance[:,:,1] == d[0]) & (instance[:,:,2] == d[1]))
        dynmask[mask] = i
        i += 1
    dyn_masks.append(dynmask)
        
dyn_masks = np.array(dyn_masks)
np.savez(outdir + 'dyn_masks.npz', dyn_masks)

In [ ]:
outdir = '/home/tristram/data/carla/25_dyn2/'
dmasks = np.load(outdir+'dyn_masks.npz', allow_pickle=True)['arr_0']
for j in range(int(len(dmasks)/5)):
    for i in range(int(len(dmasks)/5)):
        if i > 0:
            dmasks[j+i*5][prev] = prev_mask[prev]
            prev = (dmasks[j+i*5] > 0)
            prev_mask = dmasks[j+i*5]
        else:
            prev = (dmasks[j] > 0)
            prev_mask = dmasks[j]
        plt.figure()
        plt.imshow(dmasks[j+i*5])
        plt.show()

In [ ]:
masks = np.load('/home/tristram/data/carla/25_no_bkgd/2D_boxes.npz', allow_pickle=True)['arr_0']
for m in masks:
    plt.figure()
    plt.imshow(m)
    plt.show()
#np.savez('/home/tristram/data/carla/50_box_1/2D_boxes.npz', masks)